# Quantum Computing with blueqat.jl

## Using module

As for now, clone the repository, include module's path and import it by "using ~".

In [1]:
include("./blueqat.jl")
using .blueqat

## Construct quantum circuit

Prepare input state.

In [2]:
input = Input(2) # Input(n) : n qubits |0> state

Input(2, Complex[0 + 0im, 0 + 0im], Gate[])

Prepare quantum gates as a list.

In [3]:
gates = [H(1), CX(1, 2)]
# gates = [H(1), H(2), CZ(1, 2), H(1), H(2)]
# gates = [H(1), H(2), CZ(1, 2), H(1), H(2), CZ(1, 2), H(1), H(2)]
# gates = [X(1), X(2), X(3)]

2-element Array{Gate,1}:
 H(1, "H")
 CX(1, 2, "CX")

Apply gates to input state.

In [4]:
apply!(input, gates)

Input(2, Complex[0 + 0im, 0 + 0im], Gate[H(1, "H"), CX(1, 2, "CX")])

## Choice a device

Select a device.  
It is actually a simulator backend. Other simulators or hardware resources can be added to devices.

### "UndirectedGraph" simulator

In [5]:
device = get_device("UndirectedGraph")

UndirectedGraphModel()

## Quantum circuit execution

In [6]:
state = execute(input, device)

4-element Array{Complex{Float64},1}:
 0.7071067811865474 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
 0.7071067811865474 + 0.0im

Get amplitude of specified basis.

To measure amplitude of |00>, set option parameter [0, 0] (zeros(Int64, 2)).

In [7]:
state = execute(input, device, zeros(Int64, 2))

1-element Array{Complex{Float64},1}:
 0.7071067811865474 + 0.0im

### "StateVector" simulator

In [8]:
device = get_device("StateVector")

StateVectorModel(true, Complex{Float64}[0.0 + 0.0im])

In [9]:
result = execute(input, device)

StateVectorResults(Any[0.7071067811865476 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.7071067811865476 + 0.0im], Any[[0, 0]])

In [10]:
result.states

4-element Array{Any,1}:
 0.7071067811865476 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
 0.7071067811865476 + 0.0im

In [11]:
result.cregs

1-element Array{Any,1}:
 [0, 0]

With "StateVector" simulator, one can use measurement gate and get counts of measurements.

In [12]:
input = Input(3)
gates = [H(1), CX(1, 2), CX(1, 3), M(1), M(2), M(3)]
apply!(input, gates)

Input(3, Complex[0 + 0im, 0 + 0im, 0 + 0im], Gate[H(1, "H"), CX(1, 2, "CX"), CX(1, 3, "CX"), M(1, 1, 0, "M"), M(2, 2, 0, "M"), M(3, 3, 0, "M")])

In [13]:
result = execute(input, device, 100)

StateVectorResults(Any[Complex{Float64}[1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im], Complex{Float64}[1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im], Complex{Float64}[1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im], Complex{Float64}[1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im], Complex{Float64}[1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im], Complex{Float64}[1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im], Complex{Float64}[1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im], Complex{Float64}[0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 1.0000000000000

In [14]:
get_counts(result)

Dict{String,Int64} with 2 entries:
  "000" => 56
  "111" => 44

### "TensorNetworkStates" simulator

As an option, some device dependent parameters can be contained in "device".  
As an example for "TensorNetworkStates" simulator, one can set a parameter to limit bond dimension of matrix product state (MPS).

Float64: Cutoff value of SVD compression.   
Int64: Limit of SVD compression's dims.

In [15]:
device = get_device("TensorNetworkStates", 1e-6)

TensorNetworkStatesModel(true, Complex{Float64}[0.0 + 0.0im], 1.0e-6)

In [16]:
input = Input(2) # Input(n) : n qubits |0> state
gates = [H(1), CX(1, 2)]
apply!(input, gates)

Input(2, Complex[0 + 0im, 0 + 0im], Gate[H(1, "H"), CX(1, 2, "CX")])

In [17]:
result = execute(input, device)

TensorNetworkStatesResults(Complex{Float64}[0.7071067811865474 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.7071067811865474 + 0.0im], OrderedCollections.OrderedDict{Any,Any}("array 1's size" => (1, 2),"array 2's size" => (2, 1),"Num of parameters" => 8,"2^N" => 4))

In [18]:
result.states

4-element Array{Complex{Float64},1}:
 0.7071067811865474 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
 0.7071067811865474 + 0.0im

With "TensorNetworkStates", one can check amounts of parameters for MPS representation.

In [19]:
result.info

OrderedCollections.OrderedDict{Any,Any} with 4 entries:
  "array 1's size"    => (1, 2)
  "array 2's size"    => (2, 1)
  "Num of parameters" => 8
  "2^N"               => 4

## Compatibility between devices

One can execute same circuits with all devices.

In [20]:
input = Input(2) # Input(n) : n qubits |0> state
gates = [H(1), CX(1, 2)]
apply!(input, gates)

Input(2, Complex[0 + 0im, 0 + 0im], Gate[H(1, "H"), CX(1, 2, "CX")])

In [21]:
device1 = get_device("UndirectedGraph")
device2 = get_device("StateVector")
device3 = get_device("TensorNetworkStates", 1e-6)

TensorNetworkStatesModel(true, Complex{Float64}[0.0 + 0.0im], 1.0e-6)

In [22]:
result = execute(input, device1)

4-element Array{Complex{Float64},1}:
 0.7071067811865474 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
 0.7071067811865474 + 0.0im

In [23]:
result = execute(input, device2)
result.states

4-element Array{Any,1}:
 0.7071067811865476 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
 0.7071067811865476 + 0.0im

In [24]:
result = execute(input, device3)
result.states

4-element Array{Complex{Float64},1}:
 0.7071067811865474 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
 0.7071067811865474 + 0.0im